## 🔗 Open This Notebook in Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DavidLangworthy/ds4s/blob/master/Day%202%20%E2%80%93%20Fossil%20Fuels%20vs.%20Renewables.ipynb)

# ⚡ Day 2 – Exploring Energy Trends  
### Fossil Fuels vs. Renewables in a Warming World

Welcome to Day 2! Today you’ll go beyond the basics and learn how to compare **multiple variables**, build **multi-series plots**, and tell a more nuanced story about global energy trends.

Our big question:  
> **How has the global energy mix changed over time — and is renewable energy growing fast enough to fight climate change?**

Even with all the progress in solar, wind, and hydropower, fossil fuels still make up about **85% of global energy use**. Today, you’ll use data to visualize both the progress and the gap that remains.

---

## 🧭 What You'll Learn

This is the **“walk”** stage of the course. You’ll work more independently, but still with guidance and support. You’ll:

- Practice loading and reshaping real-world data
- Plot **multiple time series** in one chart
- Create a **stacked area chart** to show energy mix over time
- Use **bar charts** to compare countries
- Learn to use **Seaborn** to make your plots clearer and more beautiful
- Focus on framing a story with your visuals — not just showing data, but making it meaningful

---

## 🔧 Tools & Setup

We’ll use:
- `pandas` for data loading and manipulation
- `seaborn` and `matplotlib` for plotting
- Google Colab (as always) — no installation needed, just open and run

You’ll also see a few useful techniques like **filtering**, **grouping**, and possibly a light **list comprehension**.

---

## 📊 Dataset: Global Energy Mix 1965–2022

You’ll work with a **real dataset** compiled from BP and IEA sources, available publicly on Kaggle and other platforms. It tracks:
- Global **renewable energy share** over time
- Contributions from **hydropower, wind, and solar**
- National data for a set of countries

For this lab, we’ll focus on:
- Global trends over time
- Composition of renewable energy sources
- Cross-country comparisons

---

## 🛠️ Lab: Fossil vs. Renewable Energy

You’ll open the notebook called **“Fossil vs Renewable Energy”**, which will guide you through these projects:

### 1. Global Trends – Line Plot
You’ll start by loading the global dataset and creating a simple **line plot** showing the rise of renewables over time — from about 5% of energy in 2000 to ~15% today.

> You’ll use `seaborn.lineplot()` with nice styling and markers to make the chart pop.  
> Optional: annotate the chart with milestones (like Paris Agreement or major tech breakthroughs).

---

### 2. Energy Composition – Stacked Area Chart (Optional)
Next, you’ll explore **where renewable energy comes from** — hydro, wind, and solar. A **stacked area chart** will help you visualize how each part has grown.

> This creates a rich, colorful graphic that shows how solar and wind are catching up to hydropower.  
> You’ll see the energy mix **diversify over time**, not just grow.

We provide scaffolding to help you build this. You can do it! And if not, the solution notebook shows what’s possible.

---

### 3. Country Comparison – Bar Chart
Then, you’ll look at how different countries are doing. You’ll use a prepared dataset for a recent year (like 2021) to compare **renewable energy share by country**.

> You’ll create a **horizontal bar chart** using `seaborn.barplot()` or `barh()`.

You might compare:
- Top 5 leaders in renewable share (e.g., Norway, Brazil)
- Big emitters with low renewables (e.g., US, China, India)

> The story: Some countries are way ahead. Others have a long way to go.

---

### 4. Dual-Axis Plot (Stretch Goal)
As a challenge, we’ll show how to build a **dual-axis chart** — one line for global energy demand, another for renewable share.

> This gives extra context: Even as renewables grow, **total energy use is rising too**, which makes hitting climate goals harder.

This section is mostly prepared in the starter notebook so you can focus on understanding what it shows.

---

## 🧠 Storytelling Prompts

At key points, you’ll be asked to **interpret the data**:
- How much have renewables grown?
- Who’s leading and who’s lagging?
- What does this mean for climate action?

You’ll connect your visual insights to big ideas:
- Fossil fuels still dominate emissions (~75% of global GHGs)
- Scaling up renewables is essential — not just for CO₂, but also for reducing local air pollution and health impacts

---

## 🧪 Starter vs. Solution

Just like yesterday:
- The **starter notebook** walks you through each task with scaffolding and comments (e.g., `# TODO: plot multiple time series`)
- The **solution notebook** shows polished versions of all plots — colorful, clear, and ready to be shared

---

## ✅ By the End of Today

You’ll be able to:
- Create multi-series and multi-category visualizations
- Use Seaborn to improve plot readability
- Compare global trends and country-level differences
- Build high-impact graphics that show both progress and urgency

This is your next step in becoming a data storyteller. Let’s go build some beautiful, meaningful plots.


In [ ]:
# Day 2: Fossil Fuels vs. Renewables Visualization

# Install and import required libraries
# !pip install pandas matplotlib seaborn
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- Starter Notebook elide: import the libraries you plan to use ---

# Load datasets from the local data directory
data_dir = Path.cwd() / "data"
df_total = pd.read_csv(data_dir / "01 renewable-share-energy.csv")
df_hydro = pd.read_csv(data_dir / "06 hydro-share-energy.csv")
df_wind = pd.read_csv(data_dir / "10 wind-share-energy.csv")
df_solar = pd.read_csv(data_dir / "14 solar-share-energy.csv")

# --- Starter Notebook elide: load the renewable energy datasets ---

# Filter for 'World' data
df_total_world = df_total[df_total["Entity"] == "World"]
df_hydro_world = df_hydro[df_hydro["Entity"] == "World"]
df_wind_world = df_wind[df_wind["Entity"] == "World"]
df_solar_world = df_solar[df_solar["Entity"] == "World"]

# --- Starter Notebook elide: focus on the world-level subset ---

# Merge datasets into single dataframe for plotting
df_merged = pd.DataFrame({
    "Year": df_total_world["Year"],
    "Total Renewable": df_total_world["Renewables (% equivalent primary energy)"],
    "Hydro": df_hydro_world["Hydro (% equivalent primary energy)"].values,
    "Wind": df_wind_world["Wind (% equivalent primary energy)"].values,
    "Solar": df_solar_world["Solar (% equivalent primary energy)"].values,
})

# --- Starter Notebook elide: assemble a dataframe that combines the renewable sources ---

# Plot 1: Line plot showing renewable energy share over time
plt.figure(figsize=(10, 6))
sns.lineplot(data=df_merged, x="Year", y="Total Renewable", marker="o")
plt.title("Global Renewable Energy Share (1965–2022)")
plt.ylabel("Renewable Share of Total Energy (%)")
plt.xlabel("Year")
plt.grid(True)
plt.show()

# --- Starter Notebook elide: build the line chart that tracks total renewable share ---

# Plot 2: Stacked area chart for different types of renewable energy
plt.figure(figsize=(12, 7))
plt.stackplot(
    df_merged["Year"],
    df_merged["Hydro"],
    df_merged["Wind"],
    df_merged["Solar"],
    labels=["Hydro", "Wind", "Solar"],
    colors=["#4c72b0", "#55a868", "#c44e52"],
    alpha=0.7,
)

plt.title("Composition of Renewable Energy Sources Globally (1965–2022)")
plt.ylabel("Renewable Share (%)")
plt.xlabel("Year")
plt.legend(loc="upper left")
plt.grid(True)
plt.show()

# --- Starter Notebook elide: layer the stackplot that shows the technology mix ---

# Plot 3: Bar chart comparison for latest available year
latest_year = df_merged["Year"].max()
latest_data = df_merged[df_merged["Year"] == latest_year].melt(
    id_vars="Year", value_vars=["Hydro", "Wind", "Solar"], var_name="Type", value_name="Percentage"
)

plt.figure(figsize=(8, 6))
sns.barplot(data=latest_data, x="Type", y="Percentage", palette="Set2")
plt.title(f"Renewable Energy Composition in {latest_year}")
plt.ylabel("Share of Total Energy (%)")
plt.xlabel("Renewable Type")
plt.grid(axis="y")
plt.show()

# --- Starter Notebook elide: compare the latest mix with a bar chart ---


In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt

plots_dir = Path.cwd() / "plots"
plots_dir.mkdir(parents=True, exist_ok=True)

fig = plt.gcf()
if fig and fig.axes:
    output_path = plots_dir / "day02_solution_plot.png"
    fig.savefig(output_path, dpi=300, bbox_inches="tight")
    print(f"Saved latest Matplotlib figure to {output_path}")
else:
    print("No Matplotlib figure to save.")
